In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
"""
    双低筛选策略，参考：
    https://xueqiu.com/1314783718/166213808
    https://www.jisilu.cn/question/408977
    https://www.jisilu.cn/question/273614
    
    
    注意：105元以下的转债才有足够的安全边际
"""

import bisect
import copy
import logging
import math
from statistics import mean
import pandas as pd
from jqdata import get_all_trade_days
from jqdata import bond
from datetime import datetime, timedelta

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.DEBUG)

# 低于110元的转债我们看作低估
UNDERRATE_PRICE = 110

# 聚宽的可转债数据从2018-09-13开始记录
JQDATA_BEGIN_DATE = '2018-09-13'

        
class ConvertBondBeta(object):
    """得到指定时间的可转债基本信息，包括:
        市场总量
        低估转债(<110元)市场总量
        全部转债价格算术平均值
        全部转债溢价率算术平均值
        
        指定一个时间段，算出当前转债市场指标的历史百分位
    """
    
    def __init__(self, base_date=None, history_days=365*3):
        """
        input:
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前3年
        """
        if not base_date:
            self._base_date = datetime.now().date()
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        if self._begin_date < datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d'):
            self._begin_date = datetime.strptime(JQDATA_BEGIN_DATE, '%Y-%m-%d')
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
        
    def get_bonds(self, date=None):
        """
        获取指定日期的可转债市场正常存续的转债的基本信息
        
            code: 可转债代码
            short_name: 可转债名称
            raise_fund_volume: 发行总量(元)
            current_fund_volume: 当前存量总量(元)
            price: 收盘价格
            day_market_volume: 当日市场成交总价
            convert_premium_ratio: 转股溢价率
            convert_price: 转股价格 (如果没有下修的话就是约定转股价)
            stock_price: 正股价格
            last_cash_date: 最终兑付日(到期时间)
            double_low: 转债价格+溢价率X100
            ytm: 计算方式比较复杂，暂缺失
        """
        
        if date is None:
            date = datetime.strptime(self._base_date, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(date, '%Y-%m-%d').date()
        
        df_bonds = bond.run_query(
                query(bond.CONBOND_BASIC_INFO).filter(
                                    bond.CONBOND_BASIC_INFO.bond_type_id == 703013,
                                    bond.CONBOND_BASIC_INFO.list_status_id.in_(['301001', '301099']),
                                    bond.CONBOND_BASIC_INFO.interest_begin_date < date,
                                    bond.CONBOND_BASIC_INFO.last_cash_date >= date
                                ).order_by('code').limit(10000)
            )        
    
        bond_list = []
        
        total_market = 0.0
        for index, row in df_bonds.iterrows():            
            bond_info = {
                        'code': row['code'], 
                        'short_name': row['short_name'], 
                        'stock_code': row['company_code'],
                        'convert_price': row['convert_price'],
                        'last_cash_date': row['last_cash_date']
            }
            
            if str(row['list_status_id']) == '301099':
                # issue-2: CONBOND_BASIC_INO表中数据更新不及时，需要去BOND_BASIC_INFO中确认一下
                bond_basic_info = bond.run_query(
                                query(bond.BOND_BASIC_INFO).filter(
                                    bond.BOND_BASIC_INFO.code == row['code']
                                )
                              )            
                if str(bond_basic_info['list_status_id'][0]) == '301099':
                    # 未上市
                    continue            
            
            if row['list_date'] and row['list_date'] > date:
                # 只发布了信息，还没有正式上市，暂不记入
                continue
            
            
            # 发行总量(万元)
            if not math.isnan(row['actual_raise_fund']):
                bond_info['raise_fund_volume'] = float(row['actual_raise_fund']) * 10000
            else:
                bond_info['raise_fund_volume'] = float(row['plan_raise_fund']) * 10000
            bond_info['current_fund_volume'] = bond_info['raise_fund_volume']
            
            # 转股信息
            bond_stock = bond.run_query(
                query(bond.CONBOND_DAILY_CONVERT).filter(
                                    bond.CONBOND_DAILY_CONVERT.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_CONVERT.date <= date,
                                )
            )   
            
            # 统计转股信息，如果99%转股，就代表强赎退市，暂不记入，另外要修正存量债券数目
            if not bond_stock['acc_convert_ratio'].empty:
                if bond_stock['acc_convert_ratio'].iloc[-1] >= 99.5:
                    continue
                else:
                    bond_info['current_fund_volume'] = bond_info['raise_fund_volume'] * \
                            (100.0 - bond_stock['acc_convert_ratio'].iloc[-1]) / 100.0
            
            # 先取得转股价，然后取得正股收盘价，然后计算溢价率：转股溢价=（100/转股价格）*正股收盘价-可转债收盘价）
            # 转股价如果有下修，先取得下修转股价
            if not bond_stock['convert_price'].empty:
                bond_info['convert_price'] =  float(bond_stock['convert_price'].iloc[-1])                
            
            # 当前市场收盘价格
            bond_market = bond.run_query(
                query(bond.CONBOND_DAILY_PRICE).filter(
                                    bond.CONBOND_DAILY_PRICE.code == bond_info['code'], 
                                    bond.CONBOND_DAILY_PRICE.date <= date,
                                )
            )
            try:
                bond_info['price'] =  float(bond_market['close'].iloc[-1])
                bond_info['day_market_volume'] = float(bond_market['money'].iloc[-1])
            except Exception:
                # 有部分还没有公布信息的先跳过
                bond_info['price'] =  float(row['par'])
                continue
            
            if bond_info['price'] < 1:
                # 停牌
                continue            

                
            # 获取正股价格
            df_stock_price = get_price(bond_info['stock_code'],
                                       count = 7,
                                       end_date= date,
                                       frequency='daily', 
                                       fields=['close'])
            bond_info['stock_price'] = df_stock_price['close'][-1]
            bond_info['convert_premium_ratio'] = (bond_info['price'] - 100/bond_info['convert_price']*bond_info['stock_price']) / \
                                                 (100/bond_info['convert_price']*bond_info['stock_price'])
            bond_info['double_low'] = bond_info['price'] + bond_info['convert_premium_ratio'] * 100
                      
            bond_list.append(bond_info)
            
        bond_list = sorted(bond_list, key=lambda x: x['double_low'])
        return bond_list
        
            
    def get_bonds_factors(self, bond_list=None):
        """
        获取当前时间的市场总量, 低估转债市场总量，指数平均价格，指数平均溢价率

        output:
             (total_market(元), underrate_market, avg_price, avg_premium_ratio)
        """
        
        if bond_list is None:
            bond_list = self.get_bonds()
            
        total_market = sum([bond['current_fund_volume'] for bond in bond_list])
        underrate_market = sum([bond['current_fund_volume'] for bond in bond_list if bond['price'] <= UNDERRATE_PRICE])
        avg_price = mean([bond['price'] for bond in bond_list])
        avg_premium_ratio = mean([bond['convert_premium_ratio'] for bond in bond_list])
        return (total_market, underrate_market, avg_price, avg_premium_ratio)
                        
        

In [3]:
class StockBeta(object):
    
    def __init__(self, stock_code, index_type=0, base_date=None, history_days=365*5):
        """
        input:
            index_code: 要查询指数的代码
            index_type: 1为等权重方式计算，0为按市值加权计算
            base_date: 查询时间，格式为'yyyy-MM-dd'，默认为当天
            history_days: 默认历史区间位前八年
        """
        self._stock_code = stock_code
        self._index_type = index_type
        if not base_date:
            self._base_date = datetime.now().date() - timedelta(1)
        else:
            self._base_date = datetime.strptime(base_date, '%Y-%m-%d')
            
        self._begin_date = self._base_date - timedelta(history_days)
        self._end_date = self._base_date
        
        self._begin_date = self._begin_date.strftime('%Y-%m-%d')
        self._end_date = self._end_date.strftime('%Y-%m-%d')
        self._base_date = self._base_date.strftime('%Y-%m-%d')
            
    def get_stock_beta_factor(self, day=None):
        """
        获取当前时间的pe, pb值
        
        input:
            day: datetime.date类型，如果为None，默认代表取当前时间

        output:
            (pe, pb, roe, circulating_market_cap)
        """
        if not day:
            day = datetime.strptime(self._base_date, '%Y-%m-%d')
        
        stocks = [self._stock_code]
        q = query(
            valuation.pe_ratio, valuation.pb_ratio, valuation.circulating_market_cap
        ).filter(
            valuation.code.in_(stocks)
        )

        df = get_fundamentals(q, day)

        df = df[df['pe_ratio']>0]

        if len(df)>0:
            if(self._index_type == 0):
                pe = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pe_ratio']).sum()
                pb = df['circulating_market_cap'].sum() / (df['circulating_market_cap']/df['pb_ratio']).sum()
            else:
                pe = df['pe_ratio'].size / (1/df['pe_ratio']).sum()
                pb = df['pb_ratio'].size / (1/df['pb_ratio']).sum()
            return (pe, pb, pb/pe, df['circulating_market_cap'].sum() * pow(10, 8))
        else:
            return (None, None, None, None)
        
    def get_stock_beta_history_factors(self, interval=7):
        """
        获取任意指数一段时间的历史 pe,pb 估值列表，通过计算当前的估值在历史估值的百分位，来判断当前市场的估值高低。
        由于加权方式可能不同，可能公开的估值数据有差异，但用于判断估值相对高低没有问题

        input：
            interval: 计算指数估值的间隔天数，增加间隔时间可提高计算性能

        output：
            result:  指数历史估值的 DataFrame，index 为时间，列为pe，pb,roe
        """
        all_days = get_all_trade_days()

        pes = []
        roes = []
        pbs = []
        days = []

        begin = datetime.strptime(self._begin_date, '%Y-%m-%d').date()
        end = datetime.strptime(self._end_date, '%Y-%m-%d').date()  
        i = 0
        for day in all_days:
            if(day <= begin or day >= end):
                continue

            i += 1

            if(i % interval != 0):
                continue

            pe, pb, roe, circulating_market_cap = self.get_stock_beta_factor(day)
            if pe and pb and roe:
                pes.append(pe)
                pbs.append(pb)
                roes.append(roe)
                days.append(day)

        result = pd.DataFrame({'pe':pes,'pb':pbs, 'roe':roes}, index=days)
        return result
    
    def get_quantile_of_history_factors(self, factor, history_list):
        """
            获取某个因子在历史上的百分位，比如当前PE处于历史上的70%区间，意味着历史PE有70%都在当前值之下

        input:
            factor: beta因子
            history_list: 历史估值列表, DataFrame

        output:
            quantile: 历史估值百分位 (0.7)
        """
        factors = [history_list.quantile(i / 10.0)  for i in range(11)]    
        idx = bisect.bisect(factors, factor)
        if idx < 10:
            quantile = idx - (factors[idx] - factor) / (factors[idx] - factors[idx-1])   
            return quantile / 10.0    
        else:
            return 1.0


In [4]:
DOUBLE_LOW_VALUE = 130
DOUBLE_LOW_PRICE = 105

class DLowStrategy(object):
    
    # 双低策略转债个数
    EXPECTED_ITEMS_COUNT = 30
    
    def __init__(self, bond_list, base_date):
        self._bond_list = copy.deepcopy(bond_list)
        self._base_date = datetime.strptime(base_date, "%Y-%m-%d").date()
        
        
    def _set_stock_info(self, bond_list):
        """增加正股估值信息
        
            stock_pb: pb
            stock_pe: pe
            stock_pb_quantile: pb百分位
            stock_pe_quantile: pe百分位
        """
        for bond in bond_list:
            stock = StockBeta(bond['stock_code'])
            bond['stock_pe'], bond['stock_pb'], bond['stock_roe'], bond['circulating_market_cap'] = stock.get_stock_beta_factor()
            
            if bond['stock_pe'] is None or bond['stock_pb'] is None:
                # 正股报表有问题，剔除
                if bond in self._bond_list:
                    print("剔除_{}".format(bond))
                    self._bond_list.remove(bond)
                continue
            
            history_factors = stock.get_stock_beta_history_factors()
            bond['stock_pb_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pb'], history_factors['pb'])
            bond['stock_pe_quantile'] = stock.get_quantile_of_history_factors(
                                                bond['stock_pe'], history_factors['pe'])
            bond['market_cap_ratio'] = bond['current_fund_volume'] / bond['circulating_market_cap'] * 100
            
    def _filter_last_cash_date(self, bond_list):
        """筛选掉最近到期项
        """
        filter_bond_list = filter(
            lambda x: x['last_cash_date'] - self._base_date > timedelta(360),
            bond_list
        )
        return list(filter_bond_list)
        
            
    def _filter_pb_pe_quantile(self, bond_list):
        """筛选pb, pe历史百分位，排除pe>30的标的
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x['stock_pb'] > 1.3 and  x['stock_pe_quantile'] < 0.8 and x['stock_pe'] < 30,
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_pb(self, bond_list):
        """筛选PB>1.3防止下修转股价时破净限制
        """
        self._set_stock_info(bond_list)
        filter_bond_list = filter(
            lambda x: x.get('stock_pb', None) and x['stock_pb'] > 1.3,
            bond_list
        )
        return list(filter_bond_list)

        
        
    def _filter_current_fund_volume(self, bond_list):
        """剩余规模>1亿，且<10亿元的转债
        """
        filter_bond_list = filter(
            lambda x: x['current_fund_volume'] > pow(10, 8) and x['current_fund_volume'] < pow(10, 9),
            bond_list
        )
        return list(filter_bond_list)

    def _filter_current_market_volume(self, bond_list):
        """当日市场成交额>100万
        """
        filter_bond_list = filter(
            lambda x: x['day_market_volume'] > pow(10, 6),
            bond_list
        )
        return list(filter_bond_list)
        
    def _filter_convert_premium_ratio(self, bond_list):
        """溢价率小于30%
        """
        filter_bond_list = filter(
            lambda x: x['convert_premium_ratio'] < 0.3,
            bond_list
        )
        return list(filter_bond_list)
        
        
    def _filter_double_low(self, bond_list):
        """双低小于125-130
        """
        filter_bond_list = filter(
            lambda x: x['double_low'] < DOUBLE_LOW_VALUE,
            bond_list
        )
        return list(filter_bond_list)
    
    def _filter_price(self, bond_list):
        """价格过滤
        """
        filter_bond_list = filter(
            lambda x: x['price'] < DOUBLE_LOW_PRICE,
            bond_list
        )
        return list(filter_bond_list)   
    
    def _filter_stock_marketcap(self, bond_list):
        """过滤总市值<10亿或股价<3元的转债
        """        
        filter_bond_list = filter(
            lambda x: x['circulating_market_cap'] > pow(10, 9) and x['stock_price'] > 3,
            bond_list
        )        
        return list(filter_bond_list)           
        
    def _filter_strict(self, bond_list):
        """ROE>8%且转债占比<25%
        """
        filter_bond_list = filter(
            lambda x: x['stock_roe'] > 0.08 and x['market_cap_ratio'] < 25,
            bond_list
        )
                
        return list(filter_bond_list)    
               
        
    
    def get_support_bonds(self, filter_pb=False, filter_pb_pe_quantile=False, filter_price=False, filter_strict=False):
        """
        剩余规模>1亿，且<10亿元的转债
        
        到期时间>一年
        
        当前成交额>100万元的转债
        
        溢价率小于15%,主要是防止市场下跌时杀转债溢价；正股下跌，带动转债价格下跌；溢价率低的转债安全垫更厚；
        另外要注意折价的情况，最典型的就是2020年初的英联转债，折价转债是否值得入手，这个需要仔细研究，
        
        到期税后收益率大于0的转债
        
        取双低值<DOUBLE_LOW_VALUE的转债
        
        筛选PB>1.3防止下修转股价时破净限制
        
        filter_pb_pe_quantile=True, 筛选pb, pe历史百分位
        """
        support_bond_list = self._bond_list
        
        support_bond_list = self._filter_current_fund_volume(support_bond_list)
        support_bond_list = self._filter_current_market_volume(support_bond_list)
        support_bond_list = self._filter_convert_premium_ratio(support_bond_list)
        support_bond_list = self._filter_double_low(support_bond_list)
        support_bond_list = self._filter_last_cash_date(support_bond_list)
        
        
        if filter_pb:
            print("filter pb")
            support_bond_list = self._filter_pb(support_bond_list)
        
        if filter_pb_pe_quantile:
            print("filter pb quantile")
            support_bond_list = self._filter_pb_pe_quantile(support_bond_list)
            
        if filter_price:
            support_bond_list = self._filter_price(support_bond_list)                
            
        if filter_strict:
            print("市值>10亿，正股>3")
            support_bond_list = self._filter_stock_marketcap(support_bond_list)
            print("ROE>8%，转债占比<25%")
            support_bond_list = self._filter_strict(support_bond_list)                            
            
        return support_bond_list


In [5]:
# 测试
import pandas as pd
from datetime import datetime, timedelta
from jqfactor import *

import warnings

base_date = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')
#base_date = datetime.now()strftime('%Y-%m-%d')
# 后视镜转债历史低点  '2018-10-18' 
#base_date = '2018-10-18' 

index_bond = ConvertBondBeta(base_date=base_date)
print(base_date)
print("=========================")

# 获取指定时间转债列表及统计值 
bond_list = index_bond.get_bonds(base_date)
#pd.DataFrame(bond_list)

2021-03-21


In [6]:
# 采用价格过滤
bond_list_a = DLowStrategy(bond_list, base_date).get_support_bonds(filter_pb=False, filter_pb_pe_quantile=False, filter_price=True)
pd.DataFrame(bond_list_a)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_price
0,113609,0.015205,20.34,8.864800e+08,3.781736e+07,103.340529,2026-11-24,101.820,8.864800e+08,永安转债,603776.XSHG,20.40
1,128127,0.085936,5.37,9.500000e+08,6.358246e+06,104.851584,2026-08-20,96.258,9.500000e+08,文科转债,002775.XSHE,4.76
2,123100,0.094210,15.34,3.800000e+08,6.733169e+07,110.210991,2027-02-09,100.790,3.800000e+08,朗科转债,300543.XSHE,14.13
3,123097,0.057454,9.34,3.000000e+08,1.134833e+07,110.245395,2027-01-27,104.500,3.000000e+08,美力转债,300611.XSHE,9.23
4,113601,0.144284,16.98,5.433100e+08,4.702423e+06,110.998374,2026-08-21,96.570,5.433100e+08,塞力转债,603716.XSHG,14.33
5,113610,0.115413,8.81,5.250000e+08,6.242677e+06,111.561291,2026-12-01,100.020,5.250000e+08,灵康转债,603669.XSHG,7.90
6,113591,0.115511,10.46,5.497250e+08,3.491782e+06,112.011125,2026-07-01,100.460,5.500000e+08,胜达转债,603687.XSHG,9.42
7,123085,0.192903,6.24,9.000000e+08,1.227924e+07,112.390349,2026-12-11,93.100,9.000000e+08,万顺转2,300057.XSHE,4.87
8,123004,0.113163,3.82,8.032200e+08,3.011444e+07,113.016332,2023-12-18,101.700,1.100000e+09,铁汉转债,300197.XSHE,3.49
9,128066,0.149409,9.67,4.797600e+08,3.098732e+06,113.240871,2025-04-17,98.300,4.800000e+08,亚泰转债,002811.XSHE,8.27


In [7]:
# 采用双低值过滤
bond_list_b = DLowStrategy(bond_list_a, base_date).get_support_bonds()
pd.DataFrame(bond_list_b)

,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,price,raise_fund_volume,short_name,stock_code,stock_price
0,113609,0.015205,20.34,8.864800e+08,3.781736e+07,103.340529,2026-11-24,101.820,8.864800e+08,永安转债,603776.XSHG,20.40
1,128127,0.085936,5.37,9.500000e+08,6.358246e+06,104.851584,2026-08-20,96.258,9.500000e+08,文科转债,002775.XSHE,4.76
2,123100,0.094210,15.34,3.800000e+08,6.733169e+07,110.210991,2027-02-09,100.790,3.800000e+08,朗科转债,300543.XSHE,14.13
3,123097,0.057454,9.34,3.000000e+08,1.134833e+07,110.245395,2027-01-27,104.500,3.000000e+08,美力转债,300611.XSHE,9.23
4,113601,0.144284,16.98,5.433100e+08,4.702423e+06,110.998374,2026-08-21,96.570,5.433100e+08,塞力转债,603716.XSHG,14.33
5,113610,0.115413,8.81,5.250000e+08,6.242677e+06,111.561291,2026-12-01,100.020,5.250000e+08,灵康转债,603669.XSHG,7.90
6,113591,0.115511,10.46,5.497250e+08,3.491782e+06,112.011125,2026-07-01,100.460,5.500000e+08,胜达转债,603687.XSHG,9.42
7,123085,0.192903,6.24,9.000000e+08,1.227924e+07,112.390349,2026-12-11,93.100,9.000000e+08,万顺转2,300057.XSHE,4.87
8,123004,0.113163,3.82,8.032200e+08,3.011444e+07,113.016332,2023-12-18,101.700,1.100000e+09,铁汉转债,300197.XSHE,3.49
9,128066,0.149409,9.67,4.797600e+08,3.098732e+06,113.240871,2025-04-17,98.300,4.800000e+08,亚泰转债,002811.XSHE,8.27


In [8]:
# 采用pb>1.3, pe, pb百分位筛选
bond_list_c = DLowStrategy(bond_list_a, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True)
pd.DataFrame(bond_list_c)


filter pb
剔除_{'code': '123004', 'short_name': '铁汉转债', 'stock_code': '300197.XSHE', 'convert_price': 3.82, 'last_cash_date': datetime.date(2023, 12, 18), 'raise_fund_volume': 1100000000.0, 'current_fund_volume': 803220000.0, 'price': 101.7, 'day_market_volume': 30114440.64, 'stock_price': 3.49, 'convert_premium_ratio': 0.11316332378223493, 'double_low': 113.0163323782235, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
剔除_{'code': '123024', 'short_name': '岱勒转债', 'stock_code': '300700.XSHE', 'convert_price': 18.33, 'last_cash_date': datetime.date(2024, 3, 21), 'raise_fund_volume': 210000000.0, 'current_fund_volume': 209748000.0, 'price': 103.72, 'day_market_volume': 6019314.59, 'stock_price': 16.57, 'convert_premium_ratio': 0.14736729028364506, 'double_low': 118.4567290283645, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
filter pb quantile


,circulating_market_cap,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,market_cap_ratio,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,3.814730e+09,113609,0.015205,20.34,886480000.0,37817356.00,103.340529,2026-11-24,23.238342,101.820,886480000.0,永安转债,603776.XSHG,1.3768,0.265951,7.6267,0.382909,20.40,0.180524
1,2.232210e+09,123100,0.094210,15.34,380000000.0,67331687.44,110.210991,2027-02-09,17.023488,100.790,380000000.0,朗科转债,300543.XSHE,3.5484,0.219894,25.4754,0.042638,14.13,0.139287
2,5.636180e+09,113610,0.115413,8.81,525000000.0,6242677.00,111.561291,2026-12-01,9.314820,100.020,525000000.0,灵康转债,603669.XSHG,4.1313,0.472889,27.7322,0.417558,7.90,0.148971
3,1.335890e+09,113591,0.115511,10.46,549725000.0,3491782.00,112.011125,2026-07-01,41.150469,100.460,550000000.0,胜达转债,603687.XSHG,2.4909,0.093107,28.8477,0.093110,9.42,0.086347
4,1.352390e+09,128120,0.138340,24.37,260000000.0,6019589.70,114.916027,2026-07-17,19.225223,101.082,260000000.0,联诚转债,002921.XSHE,2.3136,0.322547,26.1292,0.075330,21.64,0.088545
5,4.697730e+09,113599,0.177674,24.82,719928000.0,1700097.00,119.307421,2026-08-05,15.325019,101.540,720000000.0,嘉友转债,603871.XSHG,2.3594,0.092593,13.5692,0.118730,21.40,0.173879
6,2.072000e+09,113567,0.149177,11.46,209958000.0,1480873.00,119.807725,2026-03-04,10.133108,104.890,210000000.0,君禾转债,603617.XSHG,3.2372,0.091891,23.3107,0.063077,10.46,0.138872
7,2.306670e+09,128026,0.239890,11.45,919356000.0,9755585.92,121.988950,2023-12-13,39.856416,98.000,920000000.0,众兴转债,002772.XSHE,1.3559,0.421485,22.2410,0.191541,9.05,0.060964
8,2.972270e+09,123059,0.222313,9.91,391360860.0,3078593.39,124.111332,2026-07-15,13.167070,101.880,391400000.0,银信转债,300231.XSHE,2.7256,0.272561,22.4567,0.050938,8.26,0.121371
9,5.326250e+09,127026,0.272344,12.85,700000000.0,2939959.30,125.457430,2026-12-08,13.142455,98.223,700000000.0,超声转债,000823.XSHE,1.4483,0.166334,17.2790,0.091839,9.92,0.083819


In [9]:
# 采用最严格的过滤方法: ROE>8%，转债占流通市值<30%，
bond_list_d = DLowStrategy(bond_list_a, base_date).get_support_bonds(filter_pb=True, filter_pb_pe_quantile=True, filter_strict=True)
pd.DataFrame(bond_list_d)

filter pb
剔除_{'code': '123004', 'short_name': '铁汉转债', 'stock_code': '300197.XSHE', 'convert_price': 3.82, 'last_cash_date': datetime.date(2023, 12, 18), 'raise_fund_volume': 1100000000.0, 'current_fund_volume': 803220000.0, 'price': 101.7, 'day_market_volume': 30114440.64, 'stock_price': 3.49, 'convert_premium_ratio': 0.11316332378223493, 'double_low': 113.0163323782235, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
剔除_{'code': '123024', 'short_name': '岱勒转债', 'stock_code': '300700.XSHE', 'convert_price': 18.33, 'last_cash_date': datetime.date(2024, 3, 21), 'raise_fund_volume': 210000000.0, 'current_fund_volume': 209748000.0, 'price': 103.72, 'day_market_volume': 6019314.59, 'stock_price': 16.57, 'convert_premium_ratio': 0.14736729028364506, 'double_low': 118.4567290283645, 'stock_pe': None, 'stock_pb': None, 'stock_roe': None, 'circulating_market_cap': None}
filter pb quantile
市值>10亿，正股>3
ROE>8%，转债占比<25%


,circulating_market_cap,code,convert_premium_ratio,convert_price,current_fund_volume,day_market_volume,double_low,last_cash_date,market_cap_ratio,price,raise_fund_volume,short_name,stock_code,stock_pb,stock_pb_quantile,stock_pe,stock_pe_quantile,stock_price,stock_roe
0,3.814730e+09,113609,0.015205,20.34,886480000.0,37817356.00,103.340529,2026-11-24,23.238342,101.820,886480000.0,永安转债,603776.XSHG,1.3768,0.265951,7.6267,0.382909,20.40,0.180524
1,2.232210e+09,123100,0.094210,15.34,380000000.0,67331687.44,110.210991,2027-02-09,17.023488,100.790,380000000.0,朗科转债,300543.XSHE,3.5484,0.219894,25.4754,0.042638,14.13,0.139287
2,5.636180e+09,113610,0.115413,8.81,525000000.0,6242677.00,111.561291,2026-12-01,9.314820,100.020,525000000.0,灵康转债,603669.XSHG,4.1313,0.472889,27.7322,0.417558,7.90,0.148971
3,1.352390e+09,128120,0.138340,24.37,260000000.0,6019589.70,114.916027,2026-07-17,19.225223,101.082,260000000.0,联诚转债,002921.XSHE,2.3136,0.322547,26.1292,0.075330,21.64,0.088545
4,4.697730e+09,113599,0.177674,24.82,719928000.0,1700097.00,119.307421,2026-08-05,15.325019,101.540,720000000.0,嘉友转债,603871.XSHG,2.3594,0.092593,13.5692,0.118730,21.40,0.173879
5,2.072000e+09,113567,0.149177,11.46,209958000.0,1480873.00,119.807725,2026-03-04,10.133108,104.890,210000000.0,君禾转债,603617.XSHG,3.2372,0.091891,23.3107,0.063077,10.46,0.138872
6,2.972270e+09,123059,0.222313,9.91,391360860.0,3078593.39,124.111332,2026-07-15,13.167070,101.880,391400000.0,银信转债,300231.XSHE,2.7256,0.272561,22.4567,0.050938,8.26,0.121371
7,5.326250e+09,127026,0.272344,12.85,700000000.0,2939959.30,125.457430,2026-12-08,13.142455,98.223,700000000.0,超声转债,000823.XSHE,1.4483,0.166334,17.2790,0.091839,9.92,0.083819
8,6.247400e+09,113588,0.253708,13.36,549945000.0,7816376.00,126.530835,2026-06-17,8.802782,101.160,550000000.0,润达转债,603108.XSHG,2.0845,0.116774,24.6624,0.360452,10.78,0.084521
9,3.670730e+09,123010,0.269191,12.20,429613000.0,1188336.88,127.518111,2024-07-05,11.703748,100.599,430000000.0,博世转债,300422.XSHE,1.6561,0.026327,17.8758,0.357800,9.67,0.092645


In [10]:
bond_list_a_text = pd.DataFrame(bond_list_a).to_html()
bond_list_b_text = pd.DataFrame(bond_list_b).to_html()
bond_list_c_text = pd.DataFrame(bond_list_c).to_html()
bond_list_d_text = pd.DataFrame(bond_list_d).to_html()


# 取得几个统计值：市场总量，小于110元转债总量，价格算术平均值，溢价率算术平均值
total_market, underrate_market, avg_price, avg_premium_ratio = index_bond.get_bonds_factors(bond_list)
total_market_text = "市场总量{}亿元, 小于110元转债总量{}亿元, 价格算术平均值{}, 溢价率算术平均值{}".format(
                                                                total_market/100000000,
                                                                underrate_market/100000000,
                                                                avg_price, avg_premium_ratio
                    )

split_text = '=================================================='
print(total_market_text)

send_message_text = "{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n{}\r\n".format(
    bond_list_a_text, split_text, bond_list_b_text, bond_list_c_text, split_text, total_market_text)
#print(send_message_text)

市场总量5198.504772485299亿元, 小于110元转债总量3258.5726413465995亿元, 价格算术平均值122.90381065088758, 溢价率算术平均值0.30357336758659625
